# Mollweide Gui

In [ ]:
%pylab ipympl

In [ ]:
import os
import sys
import inspect
import logging
import ipywidgets as widgets

# if you make changes to the ppmpy module (e.g. add your analysis methods via a pull 
# request) in the https://github.com/PPMstar/PyPPM repo you may want use that 
# updated version
sys.path.insert(0, '/user/scratch14_ppmstar-lcse/fherwig/PyPPM')
from ppmpy import ppm
print(inspect.getfile(ppm))

# turn off matplotlib messages
logging.getLogger("matplotlib").setLevel(logging.CRITICAL)

The next cell will scan the ASDR data and find runs that have the required Moms and Rprof data needed for the Moms GUI. This takes a moment. If you know what you want you can just hardwire the directory variable `rprof_dir` and `moms_dir` and skip this step. Else select a run in the drop-down menu and don't forget to press the _Select Runs_ button! The scanner may find runs it thinks it can render because data files are present. The scanner does not check for physical data integrity and if the correct values are 

In [ ]:
base_dir = '/data/ASDR/PPMstar/'
root_dirs = os.listdir(base_dir)
options = []

for sub_dir in root_dirs:
    if (os.path.isdir(base_dir + sub_dir)):
        data_dirs = os.listdir(base_dir + sub_dir)
        for folder in data_dirs:
            if (os.path.isdir(base_dir + sub_dir + '/' + folder)):
                full_dir = base_dir + sub_dir + '/' + folder
                if (os.path.exists(full_dir + '/prfs') and os.path.exists(full_dir + '/myavsbq')):
                    if ((os.listdir(full_dir + '/prfs') != []) and (os.listdir(full_dir + '/myavsbq') != [])):
                        options.append(full_dir)

select_dropdown = widgets.Dropdown(options=options, description="Path Select", layout=widgets.Layout(width='450px'))
select_button = widgets.Button(description="Select Runs", disabled=False)
selected_label = widgets.Label(value="")
select_group = widgets.VBox([select_dropdown, select_button, selected_label])

def select(b):
    global rprof_dir
    global moms_dir
    rprof_dir = select_dropdown.value + '/prfs/'
    moms_dir = select_dropdown.value + '/myavsbq/'
    selected_label.value = select_dropdown.value
select_button.on_click(select)

display(select_group)

In [ ]:
# A tested case: N15 is a RAWD simulation on a 768^3 grid (Stephens et al. 2020)
rprof_dir = "/data/ASDR/PPMstar/LowZRAWD/N15-LowZRAWD-768-10x-burn-moms/prfs/"
moms_dir  = "/data/ASDR/PPMstar/LowZRAWD/N15-LowZRAWD-768-10x-burn-moms/myavsbq/"

## Mollweide Gui: How to use guide
It is advisable to start with looking at the radial profiles using the `RProf_gui.ipynb` notebook to get an overview of the run.

1) Select the path to the data you wish to examine in dropdown above\
2) Click the `Select Runs` button above\
3) Run the block below to initialize the `Mollweide Gui` for those runs of data

There are 5 sets of controls below in the `Graph Settings` tab, they are the following:
- Dump, selects the dump number of the data set
- Radius, selected the radius from the center of the start to take the shell which will be projected to the mollweide plot
- Quantity, there are 6 quantities examinable in this GUI. These are defined in `get_mollweide_data`, see `quantity =` and calculated in `get_mollweide_data` using the original 10 quantities in the decompressed Moms data. If the decompresssed Moms data have different quantities the GUI may show garbage. Currently these 10 are hardwired to be (we need to write out what variable are decompressed in a way that the MomsData class can read ... ):
| index | quantity            |
|:-----:|:--------------:     |
| 0     | x                   |
| 1     | $\vec{u_{x}}$                  |   
| 2     | $\vec{u_{y}}$             | 
|  3    | $\vec{u_{z}}$             | 
|  4    | $\lvert\vec{u_{t}}\rvert$         | 
|  5    | $\lvert\vec{u_{r}}\rvert$             | 
|  6    | $\lvert\vec{\omega}\rvert$      | 
|  7    | P              | 
|  8    | rho            | 
|  9    | fv             | 
- Log Value Range/Value Range. Log Value Range applies a limit to the maximum and minimum values logged within the selected data symetrically about the zero and then offsets by log10(min). Value Range just limits the range of unlogged data to show, this will automatically go to min/max of the selected data when switched to from log or when the quantity is switched.

There are 3 different types of controls in the `Colour Picker` tab
- Colours dropdown, this dropdown selects the number of colours to include on the colour map for the graph
- Individual colours, these pickers open up a palet when clicked on to allow the user to select the exact colour desired
- Range sliders, each with a range from 0 to 1 with the first non-greyed out colour being set to 0 and the last being set to 1. 0 is the bottom of the colour mapping bar right of the graph and 1 is the top, the user can arrange colours however they choose between these set points.

In [ ]:
#get rprof and moms data
rprof_data = ppm.RprofSet(rprof_dir)
moms_data = ppm.MomsDataSet(moms_dir, 100, rprofset=rprof_data)
# The second argument (100) in the previous call is any actually available dump
# and is only needed for initialization. Dump 100 may not be available, then 
# try a different dump, maybe check out RProf_gui notebook first.

moms_data.mollweide_gui()